In [9]:
import os
import sys
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

os.chdir('..')
from db_connection.connection import PostgresConnection
from src.df_cleaner import *
from src.df_outlier_handler import OutlierHandler



In [10]:
import warnings
warnings.filterwarnings('ignore')


# Reading Data

In [11]:
missing_values = ["n/a", "na", "undefined"]
df = pd.read_excel("C:/Users/hp/Documents/10 academy/Week 4/data/Apollo android review data.xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              100 non-null    object        
 1   userName              100 non-null    object        
 2   userImage             100 non-null    object        
 3   👍                     100 non-null    int64         
 4   reviewCreatedVersion  72 non-null     object        
 5   at                    100 non-null    datetime64[ns]
 6   replyContent          0 non-null      float64       
 7   repliedAt             0 non-null      float64       
 8   appVersion            72 non-null     object        
 9   score                 100 non-null    int64         
 10  Comments              100 non-null    object        
 11  Keywords              50 non-null     object        
 12  LDA_Category          100 non-null    object        
 13  Sentiment            

# General Statistics


In [12]:
df.size

1500

In [13]:
df.shape

(100, 15)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              100 non-null    object        
 1   userName              100 non-null    object        
 2   userImage             100 non-null    object        
 3   👍                     100 non-null    int64         
 4   reviewCreatedVersion  72 non-null     object        
 5   at                    100 non-null    datetime64[ns]
 6   replyContent          0 non-null      float64       
 7   repliedAt             0 non-null      float64       
 8   appVersion            72 non-null     object        
 9   score                 100 non-null    int64         
 10  Comments              100 non-null    object        
 11  Keywords              50 non-null     object        
 12  LDA_Category          100 non-null    object        
 13  Sentiment            

In [15]:
df.describe()

,👍,at,replyContent,repliedAt,score
count,100.000000,100,0.0,0.0,100.000000
mean,9.160000,2023-11-09 06:06:40.990000128,NaN,NaN,3.530000
min,0.000000,2023-10-09 10:44:00,NaN,NaN,1.000000
25%,0.000000,2023-10-25 18:18:54.750000128,NaN,NaN,1.750000
50%,0.000000,2023-11-09 18:23:58,NaN,NaN,5.000000
75%,1.000000,2023-11-25 15:13:10.249999872,NaN,NaN,5.000000
max,374.000000,2023-12-05 14:47:40,NaN,NaN,5.000000
std,45.394504,NaN,NaN,NaN,1.720083


# Missing Values

In [16]:
percent_missing_values(df)

The dataset contains 20.4 % missing values.


In [17]:
def missing_values_table(df):
        mis_val = df.isnull().sum()

        mis_val_percent = 100 * df.isnull().sum() / len(df)

        mis_val_dtype = df.dtypes

        mis_val_table = pd.concat([mis_val, mis_val_percent, mis_val_dtype], axis=1)

        mis_val_table_ren_columns = mis_val_table.rename(
        columns={0: 'Missing Values', 1: '% of Total Values', 2: 'Dtype'})

        mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:, 1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)

        print("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
             "There are " + str(mis_val_table_ren_columns.shape[0]) +
             " columns that have missing values.")

        return mis_val_table_ren_columns
print(f"Missing values: {missing_values_table(df)}")

Your selected dataframe has 15 columns.
There are 5 columns that have missing values.
Missing values:                       Missing Values  % of Total Values    Dtype
replyContent                     100              100.0  float64
repliedAt                        100              100.0  float64
Keywords                          50               50.0   object
reviewCreatedVersion              28               28.0   object
appVersion                        28               28.0   object


In [18]:
percent_missing_values(df)

The dataset contains 20.4 % missing values.


In [19]:
df.isnull().sum()

reviewId                  0
userName                  0
userImage                 0
👍                         0
reviewCreatedVersion     28
at                        0
replyContent            100
repliedAt               100
appVersion               28
score                     0
Comments                  0
Keywords                 50
LDA_Category              0
Sentiment                 0
Insight                   0
dtype: int64

In [20]:
df.isnull().sum().sum()

306